In [28]:

%matplotlib inline
import matplotlib.pyplot as plt

import json
import os
import json
import math
import pandas as pd
from pathlib import Path
from pymongo import MongoClient
from datetime import datetime
import time
# from matplotlib.finance import candlestick_ohlc
# from mpl_finance import candlestick_ohlc
# import matplotlib.dates as mdates
# import mplfinance as mpf
import numpy as np
    
import talib
import tushare as ts
from apps.dailyrun import bhs_rule_yt, data_prepare, print_info

from scipy.stats import skew, kurtosis


try: 
    conn = MongoClient() 
    print("Connected successfully!!!") 
    print()
except:   
    print("Could not connect to MongoDB") 

db = conn.stock
dt_db = conn.decision_tree
stock_list = db.list_collection_names()



for stock in stock_list:
    realdata = pd.read_csv(Path('data/' + stock + '.CSV').resolve())
    #     realdata['High'] realdata['Low'] realdata['Close'] realdata['Open'] realdata['Volumn']
    realdata['adx'] = talib.ADX(realdata['High'], realdata['Low'], realdata['Close'], timeperiod=14)
    #     ADXR(high, low, close, timeperiod=14)
    realdata['apo'] = talib.APO(realdata['Close'], fastperiod=12, slowperiod=26, matype=0)
    # realdata['aroon'] = talib.AROON(realdata['High'], realdata['Low'], timeperiod=14)
    #     realdata['aroo'] = AROONOSC(high, low, timeperiod=14)
    realdata['bop'] = talib.BOP(realdata['Open'], realdata['High'], realdata['Low'], realdata['Close'])
    realdata['cci'] = talib.CCI(realdata['High'], realdata['Low'], realdata['Close'], timeperiod=14)
    realdata['cmo'] = talib.CMO(realdata['Close'], timeperiod=14)
    realdata['dx'] = talib.DX(realdata['High'], realdata['Low'], realdata['Close'], timeperiod=14)
    realdata['macd'], realdata['macdsignal'],realdata['macdhist']  = talib.MACD(realdata['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

    realdata['mfi'] = talib.MFI(realdata['High'], realdata['Low'], realdata['Close'], realdata['Volume'], timeperiod=14)
    realdata['minus_di'] = talib.MINUS_DI(realdata['High'], realdata['Low'], realdata['Close'], timeperiod=14)
    realdata['minus_dm'] = talib.MINUS_DM(realdata['High'], realdata['Low'], timeperiod=14)

    realdata['mom'] = talib.MOM(realdata['Close'], timeperiod=5)
    realdata['plus_di'] = talib.PLUS_DI(realdata['High'], realdata['Low'], realdata['Close'], timeperiod=14)
    realdata['plus_dm'] = talib.PLUS_DM(realdata['High'], realdata['Low'], timeperiod=14)
    realdata['ppo'] = talib.PPO(realdata['Close'], fastperiod=12, slowperiod=26, matype=0)
    #     realdata['roc'] = talib.ROC(realdata['Close'], timeperiod=10)

    realdata['rsi'] = talib.RSI(realdata['Close'], timeperiod=14)
    realdata['k'], realdata['d'] = talib.STOCH(realdata['High'], realdata['Low'], realdata['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    realdata['j']=realdata.k*3-realdata.d*2

    #     realdata['roc'] = talib.ROC(realdata['Close'], timeperiod=10)
    realdata['ema5'] = talib.EMA(realdata['Close'], timeperiod=5)
    realdata['ema25'] = talib.EMA(realdata['Close'], timeperiod=25)


    realdata['clgtema5'] = np.where(realdata['Close'] > realdata['ema5'], 1, -1)
    # print(realdata['clgtema5'])
    realdata['ema5gtema25'] = np.where(realdata['ema5'] > realdata['ema25'], 1, -1)
    realdata['macdsiggtmacd'] = np.where(realdata['macdsignal'] > realdata['macd'], 1, -1)




    data_to_store = []
    records = db[str(stock)].find({})
#     print("we are doing: ", stock)
    for doc in records:
    #     try:
        if doc['date'] != "tomorrow" :
            data_selected = realdata.loc[realdata['Date'] == doc['date']]
            real_high = data_selected['High'].values[0]
            real_low = data_selected['Low'].values[0]
            highs = len([i for i in doc['high'] if i > real_high]) 
            lows = len([i for i in doc['low'] if i < real_low])

            index = data_selected.index[0]
    #         print(data_selected.index[0])
            if (index + 5) in realdata.index :
    #             print(realdata.iloc[index+1]['Close'])
    
                rocp1 = (realdata.iloc[index+1]['Close']-realdata.iloc[index]['Close'])*100/realdata.iloc[index]['Close']
                rocp2 = (realdata.iloc[index+2]['Close']-realdata.iloc[index]['Close'])*100/realdata.iloc[index]['Close']
                rocp3 = (realdata.iloc[index+3]['Close']-realdata.iloc[index]['Close'])*100/realdata.iloc[index]['Close']
                rocp4 = (realdata.iloc[index+4]['Close']-realdata.iloc[index]['Close'])*100/realdata.iloc[index]['Close']
                rocp5 = (realdata.iloc[index+5]['Close']-realdata.iloc[index]['Close'])*100/realdata.iloc[index]['Close']
    #             print(rocp1)
    
                record = {
                    
                    "_id": stock+"_"+doc['date'],
                    'stock' : stock,
                    'date' : doc['date'],
                    'adx' : data_selected['adx'].values[0],
                    'rsi': data_selected['rsi'].values[0],
                    'apo' : data_selected['apo'].values[0],
                    'bop' : data_selected['bop'].values[0],
                    'cci' : data_selected['cci'].values[0],
                    'cmo' : data_selected['cmo'].values[0],
                    'dx' : data_selected['dx'].values[0],
                    'mfi' : data_selected['mfi'].values[0],
                    'minus_di' : data_selected['minus_di'].values[0],
                    'minus_dm' : data_selected['minus_dm'].values[0],
                    'mom' : data_selected['mom'].values[0],
                    'plus_di' : data_selected['plus_di'].values[0],
                    'plus_dm' : data_selected['plus_dm'].values[0],
                    'ppo' : data_selected['ppo'].values[0],
                    'j' : data_selected['j'].values[0],
                    'macdhist':data_selected['macdhist'].values[0],
                    'highs' : int(highs),
                    'lows' : int(lows),
                    'clgtema5' : int(data_selected['clgtema5'].values[0]),
                    'ema5gtema25' : int(data_selected['ema5gtema25'].values[0]),
                    'macdsiggtmacd' : int(data_selected['macdsiggtmacd'].values[0]),
                    'rocp1' : rocp1,
                    'rocp2' : rocp2,
                    'rocp3' : rocp3,
                    'rocp4' : rocp4,
                    'rocp5' : rocp5,
                    'up1': np.where(rocp1>0, 1, 0).item(0),
                    'up2': np.where(rocp2>0, 1, 0).item(0),
                    'up3': np.where(rocp3>0, 1, 0).item(0),
                    'up4': np.where(rocp4>0, 1, 0).item(0),
                    'up5': np.where(rocp5>0, 1, 0).item(0)

                }
                query = { "_id": record['_id'] }
                result = dt_db['data'].find_one(query)

                if result == None :
                    print("insert ", stock, " ", date)
                    dt_db['data'].insert_one(record)
                else:
                    print("update ", stock, " ", date)
                    dt_db['data'].update_one(query, record)
    
    
    
    
    

Connected successfully!!!

we are doing:  002751
we are doing:  603059
we are doing:  603538
we are doing:  603567
we are doing:  002881
we are doing:  001914
we are doing:  603337
we are doing:  002414
we are doing:  600548
we are doing:  600998
we are doing:  603239
we are doing:  603612
we are doing:  002182
we are doing:  600455
we are doing:  002393
we are doing:  000070
we are doing:  600562
we are doing:  688138
we are doing:  603045
we are doing:  000096
we are doing:  002602
we are doing:  600377
we are doing:  002511
we are doing:  603688
we are doing:  603801
we are doing:  002853
we are doing:  002780
we are doing:  002001
we are doing:  600580
we are doing:  600288
we are doing:  603636
we are doing:  603506
we are doing:  603985
we are doing:  000037
we are doing:  600732
we are doing:  603359
we are doing:  603199
we are doing:  603496
we are doing:  600346
we are doing:  600671
we are doing:  000952
we are doing:  000501
we are doing:  000988
we are doing:  603665
we ar

we are doing:  688098
we are doing:  600267
we are doing:  002115
we are doing:  002857
we are doing:  600183
we are doing:  600703
we are doing:  600764
we are doing:  002192
we are doing:  603486
we are doing:  603313
we are doing:  002065
we are doing:  600329
we are doing:  600572
we are doing:  000553
we are doing:  603035
we are doing:  603118
we are doing:  002335
we are doing:  002422
we are doing:  002856
we are doing:  002597
we are doing:  002317
we are doing:  600598
we are doing:  000401
we are doing:  600630
we are doing:  002851
we are doing:  603617
we are doing:  000819
we are doing:  002332
we are doing:  002725
we are doing:  002824
we are doing:  000800
we are doing:  000810
we are doing:  600071
we are doing:  603189
we are doing:  002023
we are doing:  002735
we are doing:  603876
we are doing:  601877
we are doing:  000930
we are doing:  603320
we are doing:  600837
we are doing:  002250
we are doing:  688007
we are doing:  002862
we are doing:  600862
we are doi

In [15]:
import pandas as pd
from pymongo import MongoClient
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
try: 
    conn = MongoClient() 
    print("Connected successfully!!!") 
    print()
except:   
    print("Could not connect to MongoDB") 
dt_db = conn.decision_tree

data = dt_db['data'].find({})
df =  pd.DataFrame(list(data))
df.reset_index(drop=True, inplace=True)
# print(df)
# pd.DataFrame(X).fillna()

# print(data_array)

X = df[['adx',
        'rsi',
        'apo',
        'bop',
        'cci',
        'cmo',
        'dx',
        'mfi',
        'minus_di',
        'minus_dm',
        'mom',
        'plus_di',
        'plus_dm',
        'ppo',
        'j',
        'macdhist',
        'clgtema5',
        'ema5gtema25',
        'macdsiggtmacd']]

feature_names = np.array(['adx',
                        'rsi',
                        'apo',
                        'bop',
                        'cci',
                        'cmo',
                        'dx',
                        'mfi',
                        'minus_di',
                        'minus_dm',
                        'mom',
                        'plus_di',
                        'plus_dm',
                        'ppo',
                        'j',
                        'macdhist',
                        'clgtema5',
                        'ema5gtema25',
                        'macdsiggtmacd'])


# X = df[['adx',
# 'apo',
# 'bop',
# 'cci',
# 'cmo',
# 'dx',
# 'mfi',
# 'minus_di',
# 'minus_dm',
# 'mom',
# 'plus_di',
# 'plus_dm',
# 'ppo',
# 'j']]

# feature_names = np.array(['adx',
# 'apo',
# 'bop',
# 'cci',
# 'cmo',
# 'dx',
# 'mfi',
# 'minus_di',
# 'minus_dm',
# 'mom',
# 'plus_di',
# 'plus_dm',
# 'ppo',
# 'j'])



y=df['up1']
# print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_train = X_train.fillna(X_train.mean())
y_train = y_train.fillna(y_train.mean())
X_test = X_test.fillna(y_train.mean())
y_test = y_test.fillna(y_train.mean())

# model=RandomForestClassifier(n_estimators=100)
model=RandomForestClassifier(n_estimators=3000, bootstrap=True, random_state = 300)
# # #Train the model using the training sets y_pred=clf.predict(X_test)
model.fit(X_train, y_train)

import pickle
filename = 'randomforestclassifier_up1.sav'
pickle.dump(model, open(filename, 'wb'))

y_pred=model.predict(X_test)
# print(y_pred)
# print(y_test)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print()
import pandas as pd
feature_imp = pd.Series(model.feature_importances_,index=feature_names).sort_values(ascending=False)
print(feature_imp)

Connected successfully!!!

Accuracy: 0.5796165068778658

bop              0.069045
j                0.067290
mfi              0.065297
macdhist         0.064078
cci              0.063466
ppo              0.062924
minus_di         0.062458
adx              0.062246
apo              0.061709
mom              0.060869
plus_di          0.060543
minus_dm         0.059807
dx               0.058287
plus_dm          0.057634
rsi              0.056677
cmo              0.056440
clgtema5         0.004281
macdsiggtmacd    0.003558
ema5gtema25      0.003391
dtype: float64


In [5]:

import pandas as pd
from pymongo import MongoClient
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics
try: 
    conn = MongoClient() 
    print("Connected successfully!!!") 
    print()
except:   
    print("Could not connect to MongoDB") 
dt_db = conn.decision_tree

data = dt_db['data'].find({})
df =  pd.DataFrame(list(data))
df.reset_index(drop=True, inplace=True)
# print(df)
# pd.DataFrame(X).fillna()

# print(data_array)

X = df[['adx',
        'rsi',
        'apo',
        'bop',
        'cci',
        'cmo',
        'dx',
        'mfi',
        'minus_di',
        'minus_dm',
        'mom',
        'plus_di',
        'plus_dm',
        'ppo',
        'j',
        'macdhist',
        'clgtema5',
        'ema5gtema25',
        'macdsiggtmacd']]

feature_names = np.array(['adx',
                        'rsi',
                        'apo',
                        'bop',
                        'cci',
                        'cmo',
                        'dx',
                        'mfi',
                        'minus_di',
                        'minus_dm',
                        'mom',
                        'plus_di',
                        'plus_dm',
                        'ppo',
                        'j',
                        'macdhist',
                        'clgtema5',
                        'ema5gtema25',
                        'macdsiggtmacd'])

y=df['rocp4']
# print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_train = X_train.fillna(X_train.mean())
y_train = y_train.fillna(y_train.mean())
X_test = X_test.fillna(y_train.mean())
y_test = y_test.fillna(y_train.mean())


model = RandomForestRegressor(n_estimators=2000, bootstrap=True, random_state = 200)
# n_estimators = 100, criterion = 'mse', max_depth = None, 
#                                min_samples_split = 2, min_samples_leaf = 1
model.fit(X_train, y_train)
y_pred=model.predict(X_test)

# Model Accuracy, how often is the classifier correct?
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
errors = abs(y_pred - y_test)
# mape = 100 * np.mean(errors / y_test)
# print(mape)
# accuracy = 100 - mape
print('Model Performance')
print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
print("Performance (R^2): ",model.score(X_test, y_test))
# print('Accuracy = {:0.2f}%.'.format(accuracy))


import pickle
filename = 'randomforestregerssor_rocp4.sav'
pickle.dump(model, open(filename, 'wb'))

feature_imp = pd.Series(model.feature_importances_,index=feature_names).sort_values(ascending=False)
print(feature_imp)

Connected successfully!!!

Model Performance
Average Error: 4.4262 degrees.
Performance (R^2):  0.14108510666939866
minus_dm         0.084210
macdhist         0.076041
mfi              0.075693
cci              0.074131
adx              0.070324
plus_di          0.068117
ppo              0.068010
mom              0.067117
bop              0.065285
j                0.064029
plus_dm          0.058581
apo              0.057475
minus_di         0.055940
dx               0.046398
rsi              0.031324
cmo              0.031221
clgtema5         0.002623
ema5gtema25      0.001782
macdsiggtmacd    0.001698
dtype: float64


In [1]:
import pandas as pd
from pymongo import MongoClient
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics
from pathlib import Path
import talib
import pickle

stock = "000636"

startDate = "2019-05-20"
endDate = "2020-03-30"

realdata = pd.read_csv(Path('data/' + stock + '.CSV').resolve())
#     realdata['High'] realdata['Low'] realdata['Close'] realdata['Open'] realdata['Volumn']

realdata['adx'] = talib.ADX(realdata['High'], realdata['Low'], realdata['Close'], timeperiod=14)
#     ADXR(high, low, close, timeperiod=14)

realdata['apo'] = talib.APO(realdata['Close'], fastperiod=12, slowperiod=26, matype=0)
# realdata['aroon'] = talib.AROON(realdata['High'], realdata['Low'], timeperiod=14)
#     realdata['aroo'] = AROONOSC(high, low, timeperiod=14)

realdata['bop'] = talib.BOP(realdata['Open'], realdata['High'], realdata['Low'], realdata['Close'])
realdata['cci'] = talib.CCI(realdata['High'], realdata['Low'], realdata['Close'], timeperiod=14)
realdata['cmo'] = talib.CMO(realdata['Close'], timeperiod=14)
realdata['dx'] = talib.DX(realdata['High'], realdata['Low'], realdata['Close'], timeperiod=14)
realdata['macd'], realdata['macdsignal'],realdata['macdhist']  = talib.MACD(realdata['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

realdata['mfi'] = talib.MFI(realdata['High'], realdata['Low'], realdata['Close'], realdata['Volume'], timeperiod=14)
realdata['minus_di'] = talib.MINUS_DI(realdata['High'], realdata['Low'], realdata['Close'], timeperiod=14)
realdata['minus_dm'] = talib.MINUS_DM(realdata['High'], realdata['Low'], timeperiod=14)

realdata['mom'] = talib.MOM(realdata['Close'], timeperiod=5)
realdata['plus_di'] = talib.PLUS_DI(realdata['High'], realdata['Low'], realdata['Close'], timeperiod=14)
realdata['plus_dm'] = talib.PLUS_DM(realdata['High'], realdata['Low'], timeperiod=14)
realdata['ppo'] = talib.PPO(realdata['Close'], fastperiod=12, slowperiod=26, matype=0)
#     realdata['roc'] = talib.ROC(realdata['Close'], timeperiod=10)

realdata['rsi'] = talib.RSI(realdata['Close'], timeperiod=14)
realdata['k'], realdata['d'] = talib.STOCH(realdata['High'], realdata['Low'], realdata['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
realdata['j']=realdata.k*3-realdata.d*2

#     realdata['roc'] = talib.ROC(realdata['Close'], timeperiod=10)
realdata['ema5'] = talib.EMA(realdata['Close'], timeperiod=5)
realdata['ema25'] = talib.EMA(realdata['Close'], timeperiod=25)

realdata['clgtema5'] = np.where(realdata['Close'] > realdata['ema5'], 1, -1)
# print(realdata['clgtema5'])
realdata['ema5gtema25'] = np.where(realdata['ema5'] > realdata['ema25'], 1, -1)
realdata['macdsiggtmacd'] = np.where(realdata['macdsignal'] > realdata['macd'], 1, -1)

selected_data = realdata.loc[(realdata["Date"]>=startDate) & (realdata["Date"]<=endDate)]
X = selected_data[['adx','rsi','apo','bop','cci','cmo','dx','mfi','minus_di','minus_dm','mom','plus_di','plus_dm','ppo','j','macdhist','clgtema5','ema5gtema25','macdsiggtmacd']]

filename = 'decisiontree/randomforestclassifier_up1.sav'
model_up1 = pickle.load(open(filename, 'rb'))
selected_data["up1"] = model_up1.predict(X)

filename = 'decisiontree/randomforestclassifier_up2.sav'
model_up2 = pickle.load(open(filename, 'rb'))
selected_data["up2"] = model_up2.predict(X)

filename = 'decisiontree/randomforestclassifier_up3.sav'
model_up3 = pickle.load(open(filename, 'rb'))
selected_data["up3"] = model_up3.predict(X)

filename = 'decisiontree/randomforestclassifier_up4.sav'
model_up4 = pickle.load(open(filename, 'rb'))
selected_data["up4"] = model_up4.predict(X)

filename = 'decisiontree/randomforestclassifier_up5.sav'
model_up5 = pickle.load(open(filename, 'rb'))
selected_data["up5"] = model_up5.predict(X)


filename = 'decisiontree/randomforestregerssor_rocp1.sav'
model_rocp1 = pickle.load(open(filename, 'rb'))
selected_data["rocp1"] = model_rocp1.predict(X)

filename = 'decisiontree/randomforestregerssor_rocp2.sav'
model_rocp2 = pickle.load(open(filename, 'rb'))
selected_data["rocp2"] = model_rocp2.predict(X)

filename = 'decisiontree/randomforestregerssor_rocp3.sav'
model_rocp3 = pickle.load(open(filename, 'rb'))
selected_data["rocp3"] = model_rocp3.predict(X)

filename = 'decisiontree/randomforestregerssor_rocp4.sav'
model_rocp4 = pickle.load(open(filename, 'rb'))
selected_data["rocp4"] = model_rocp4.predict(X)

filename = 'decisiontree/randomforestregerssor_rocp5.sav'
model_rocp5 = pickle.load(open(filename, 'rb'))
selected_data["rocp5"] = model_rocp5.predict(X)



selected_data_to_save = selected_data[['Date', 'Open', 'High', 'Low', 'Close', 'up1', 'up2', 'up3', 'up4', 'up5','rocp1','rocp2','rocp3','rocp4','rocp5']]

data_to_save = selected_data_to_save.to_dict('records')

try: 
    conn = MongoClient() 
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 

# database 
db_randomforest_anaylize = conn.randomforest_anaylize 
# print(data_to_save)

if stock in db_randomforest_anaylize.list_collection_names() :
    db_randomforest_anaylize[str(stock)].insert_many(data_to_save)
else:
    try:
        db_randomforest_anaylize.create_collection(str(stock))
    except:
        print("already have")
        
    db_randomforest_anaylize[str(stock)].insert_many(data_to_save)






/Users/kevin_gwdong/djangoproject/tensorflow/tensorenv/lib/python3.6/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kevin_gwdong/djangoproject/tensorflow/tensorenv/lib/python3.6/site-packages/ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kevin_gwdong/djangoproject/tensorflow/tensorenv/lib/python3.6/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

Connected successfully!!!


In [1]:
import pandas as pd
from pymongo import MongoClient
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from plots.anaylize_plot import days
from apps.dailyrun import print_info, bhs_rule_yt, data_prepare
stock = "000636"

try: 
    conn = MongoClient() 
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 

# database 
db_randomforest_anaylize = conn.randomforest_anaylize 

data = db_randomforest_anaylize[str(stock)].find({})

# Expand the cursor and construct the DataFrame
df =  pd.DataFrame( list(data) )
df = df.sort_values(by=['Date'], ascending=True)

for index, row in df.iterrows():
    try:
        
        date = row['Date']
        today, yesterday, yesterday1 = days(stock, date)
    #     print(today, yesterday, yesterday1)
        today_data = data_prepare(stock, "no", today)
        yesterday_data = data_prepare(stock, "no", yesterday)
        position_today, today_info = bhs_rule_yt(yesterday_data, today_data)
#         print_info(today_info)
        if ["低位趋势开始反转，开始走强，考虑进入 +5"] in today_info and ["做空存在，不要参与 -8"] not in today_info :
            print(row['Date'], " 走强")
            print("day1 ", row['up1'], " day2 ", row['up2'], " day3 ", row['up3'], " day4 ", row['up4'], " day5 ", row['up5'], " ", )
            print("today close ", row['Close'])
            print("day1 ", (1+row['rocp1']/100)*row['Close'], " day2 ", (1+row['rocp2']/100)*row['Close'], " day3 ", (1+row['rocp3']/100)*row['Close'], " day4 ", (1+row['rocp4']/100)*row['Close'], " day5 ", (1+row['rocp5']/100)*row['Close'], " ", )
            print()
            print('================================================================================')
            print()
        if ["高位趋势反转，开始走弱，马上离开 -5"] in today_info :
            print(row['Date'], " 走弱")
            print("day1 ", row['up1'], " day2 ", row['up2'], " day3 ", row['up3'], " day4 ", row['up4'], " day5 ", row['up5'], " ", )
            print("today close ", row['Close'])
            print("day1 ", (1+row['rocp1']/100)*row['Close'], " day2 ", (1+row['rocp2']/100)*row['Close'], " day3 ", (1+row['rocp3']/100)*row['Close'], " day4 ", (1+row['rocp4']/100)*row['Close'], " day5 ", (1+row['rocp5']/100)*row['Close'], " ", )
            print()
            print('================================================================================')
            print()

    except:
        print(row['Date'], " high low not ready")
    
    
    
# today, yesterday, yesterday1 = days(code, "tomorrow")

# #         print(today, yesterday, yesterday1)
# today_data = data_prepare(code, "no", today)
# yesterday_data = data_prepare(code, "no", yesterday)
# position_today, today_info = bhs_rule_yt(yesterday_data, today_data)

# if ["低位趋势开始反转，开始走强，考虑进入 +5"] in today_info and ["做空存在，不要参与 -8"] not in today_info:


# print(df)


Connected successfully!!!
000636   风华高科   2019-05-20  not ready, please do optimizer
2019-05-20  high low not ready
000636   风华高科   2019-05-21  not ready, please do optimizer
2019-05-21  high low not ready
000636   风华高科   2019-05-22  not ready, please do optimizer
2019-05-22  high low not ready
000636   风华高科   2019-05-23  not ready, please do optimizer
2019-05-23  high low not ready
000636   风华高科   2019-05-24  not ready, please do optimizer
2019-05-24  high low not ready
000636   风华高科   2019-05-27  not ready, please do optimizer
2019-05-27  high low not ready
000636   风华高科   2019-05-28  not ready, please do optimizer
2019-05-28  high low not ready
000636   风华高科   2019-05-29  not ready, please do optimizer
2019-05-29  high low not ready
000636   风华高科   2019-05-30  not ready, please do optimizer
2019-05-30  high low not ready
000636   风华高科   2019-05-31  not ready, please do optimizer
2019-05-31  high low not ready
000636   风华高科   2019-06-03  not ready, please do optimizer
2019-06-03  hig

2019-08-27  走弱
day1  0  day2  1  day3  0  day4  0  day5  0  
today close  12.83
day1  12.342278959477005  day2  12.657312173544598  day3  12.6816921330414  day4  12.54343920328206  day5  13.01385324033747  


2019-08-29  走强
day1  0  day2  1  day3  1  day4  1  day5  1  
today close  13.27
day1  12.808991505475054  day2  14.021941176641239  day3  15.102805248880847  day4  14.10530300530219  day5  13.645734676069663  


2019-09-02  走弱
day1  0  day2  1  day3  1  day4  1  day5  1  
today close  12.4
day1  12.858354011309428  day2  13.236731617120336  day3  12.456627128813912  day4  13.294085418486585  day5  13.88241653578468  


2019-09-03  走强
day1  1  day2  0  day3  0  day4  1  day5  0  
today close  13.12
day1  13.477484539779969  day2  13.382520819377572  day3  13.477904304479637  day4  13.242006392174604  day5  14.152762706157707  


2019-09-09  走强
day1  0  day2  1  day3  0  day4  0  day5  0  
today close  14.54
day1  14.619037879499823  day2  14.419718980968232  day3  14.40221843322824

2019-12-11  走弱
day1  1  day2  1  day3  1  day4  1  day5  1  
today close  14.07
day1  14.112681148824397  day2  14.178999184396485  day3  14.700213634810721  day4  14.172195294668462  day5  14.138810692294312  


2019-12-18  走弱
day1  0  day2  0  day3  0  day4  0  day5  1  
today close  14.81
day1  14.588660346295097  day2  14.697600585471493  day3  14.420017969929269  day4  14.500164588504884  day5  14.972620158040343  


2019-12-25  走强
day1  0  day2  0  day3  0  day4  1  day5  1  
today close  15.17
day1  15.345670835928699  day2  14.937477432615418  day3  15.180717597331677  day4  15.706259078907234  day5  15.462763340430412  


2019-12-26  走弱
day1  1  day2  1  day3  0  day4  1  day5  1  
today close  15.11
day1  14.899983235163713  day2  15.197144227201768  day3  15.013716541068627  day4  15.531282955907717  day5  15.373314962372712  


2019-12-30  走弱
day1  0  day2  1  day3  1  day4  1  day5  1  
today close  15.15
day1  15.175859058778695  day2  15.402961391040929  day3  16.5633088